# IEEE Fraud Detection

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Carregando os datasets

In [3]:
df_train_id = pd.read_csv("train_identity.csv")
df_train_tr = pd.read_csv("train_transaction.csv")
df_test_id = pd.read_csv("test_identity.csv")
df_test_tr = pd.read_csv("train_transaction.csv")

In [4]:
df_train_id.head(0)

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo


In [5]:
df_train_tr.head(0)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339


In [28]:
df_train = pd.concat([df_train_id, df_train_tr], ignore_index=True)
df_test = pd.concat([df_test_id, df_test_tr], ignore_index=True)

In [35]:
# Retirando a coluna isFraud para podermos fazer a predição desta
df_test.drop(columns=['isFraud'], axis=1, inplace=True)

In [36]:
print(df_train.shape)
print(df_test.shape)

(734773, 434)
(732447, 433)


In [37]:
df_train.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
df_train.columns.values

array(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05',
       'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12',
       'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19',
       'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26',
       'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33',
       'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain',
       'R_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
       'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4',
       'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14',
       'D15', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V1',
       'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14

In [39]:
df_test.head()

,TransactionID,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3663586,-45.0,280290.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663588,0.0,3579.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663597,-5.0,185210.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663601,-45.0,252944.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663602,-95.0,328680.0,NaN,NaN,7.0,-33.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df_test.columns.values

array(['TransactionID', 'id-01', 'id-02', 'id-03', 'id-04', 'id-05',
       'id-06', 'id-07', 'id-08', 'id-09', 'id-10', 'id-11', 'id-12',
       'id-13', 'id-14', 'id-15', 'id-16', 'id-17', 'id-18', 'id-19',
       'id-20', 'id-21', 'id-22', 'id-23', 'id-24', 'id-25', 'id-26',
       'id-27', 'id-28', 'id-29', 'id-30', 'id-31', 'id-32', 'id-33',
       'id-34', 'id-35', 'id-36', 'id-37', 'id-38', 'DeviceType',
       'DeviceInfo', 'TransactionDT', 'TransactionAmt', 'ProductCD',
       'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1',
       'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain', 'C1',
       'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7',
       'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2',
       'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V1', 'V2', 'V3', 'V4',
       'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14',
       'V15', '

In [48]:
# Variável alvo
target = df_train['isFraud'].value_counts()
target

0.0    569877
1.0     20663
Name: isFraud, dtype: int64

#### Avaliando o tipo de aparelho em que a transação foi realizada

In [41]:
devicetype_freq = pd.DataFrame.from_dict(df_train['DeviceType'].value_counts(normalize=True, dropna=False))

devicetype_freq.rename(columns={'DeviceType': 'Freq'}, inplace=True)
devicetype_freq

,Freq
NaN,0.808363
desktop,0.115907
mobile,0.075731


#### Avaliando o sistema operacional em que a transação foi realizada

In [43]:
dev_freq = pd.DataFrame.from_dict(df_train['DeviceInfo'].value_counts(normalize=True, dropna=False))

dev_freq.rename(columns={'DeviceInfo': 'Freq'}, inplace=True)
dev_freq = dev_freq[dev_freq['Freq'] >= 0.1/100]

dev_freq

,Freq
NaN,0.838500
Windows,0.064948
iOS Device,0.026923
MacOS,0.017111
Trident/7.0,0.010126
rv:11.0,0.002587
rv:57.0,0.001309


- Aproximadamente 83% dos dados são nulos

#### Avaliando a bandeira do cartão de crédito

In [44]:
cc_freq = pd.DataFrame.from_dict(df_train['card4'].value_counts(normalize=True, dropna=False))

cc_freq.rename(columns={'card4': 'Freq'}, inplace=True)
cc_freq 

,Freq
visa,0.523654
mastercard,0.257518
NaN,0.198442
american express,0.011334
discover,0.009052


In [12]:
df_train[['card4', 'isFraud']].groupby(['card4'], as_index=True).mean()

,isFraud
card4,
american express,0.028698
discover,0.077282
mastercard,0.034331
visa,0.034756


,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,V331,V332,V333,V334,V335,V336,V337,V338,V339,TransacionDT
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(2986999.999, 3090506.4]"
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(2986999.999, 3090506.4]"
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(2986999.999, 3090506.4]"
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(2986999.999, 3090506.4]"
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(2986999.999, 3090506.4]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734768,3577535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(3455174.6, 3577539.0]"
734769,3577536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(3455174.6, 3577539.0]"
734770,3577537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(3455174.6, 3577539.0]"
734771,3577538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(3455174.6, 3577539.0]"
